# Plate Mapper
Reads input excel file and calculates number of reactions

In [15]:
# Import the necessary libraries, including os and pandas
import os
import pandas as pd
import math

pd.set_option('display.width', 1000)

def mapped_reactions(reactions):
    
    if reactions % 2 == 0:
        return 0
    else:
        return 1

def uL_for_10ug(ng_concentration):
    ug_concen= ng_concentration / 1000
    
    max_sample_limit = round(10/ug_concen,1)
    
    if max_sample_limit > 25:
        max_sample_limit = 25
    
    return max_sample_limit

# Load the Excel file 'input.xlsx' into a DataFrame
input_df = pd.read_excel('input.xlsx', engine='openpyxl')

input_df.index =input_df.index + 1

input_df['ug DNA have'] = input_df['Volume'] * (input_df['Concentration nanograms'] / 1000)

input_df['ug of DNA needed'] = (input_df['number of guides'].astype(float) * 0.006 * input_df['Coverage']) / 1000

#limit each reaction to 10ug of DNA max
input_df['sample_rxn_vol'] = input_df['Concentration nanograms'].apply(uL_for_10ug)

#Calculate water dilution if needed
input_df['water_dilution'] = 25 - input_df['sample_rxn_vol']

#Round up to nearest whole number
input_df['number of reactions'] = (((input_df['ug of DNA needed'] / input_df['ug DNA have']) * input_df['Volume']) /25).apply(lambda x: math.ceil(x))

#Add addtional reaction to odd numbers so that plate maps are evenly spaced
input_df['empty reactions'] = input_df['number of reactions'].apply(mapped_reactions)


print(input_df)

#has the extra reaction
rxn_df = input_df[['Sample','number of reactions', 'empty reactions']].copy()


rxn_list = rxn_df.values.tolist()

    Sample  Volume  Concentration nanograms  Coverage  number of guides  ug DNA have  ug of DNA needed  sample_rxn_vol  water_dilution  number of reactions  empty reactions
1  control     150                     1500       400              5000       225.00              12.0             6.7            18.3                    1                1
2    test1     250                      100       400              5000        25.00              12.0            25.0             0.0                    5                1
3    test2     175                      450       400              5000        78.75              12.0            22.2             2.8                    2                0
4    test3     300                      150       400              5000        45.00              12.0            25.0             0.0                    4                0
5    test4    8000                       25       400              5000       200.00              12.0            25.0             0.0 

Create master list of samples and reactions to be formatted into map.  Create 'empty' spacers during this step

In [16]:
print(rxn_list)

sample_rxn_list = []
for sample in rxn_list:
    for rxn in range(sample[1]):
        sample_rxn_list.append(sample[0])
    if sample[2] == 1:
        sample_rxn_list.append('empty')

print(sample_rxn_list)

[['control', 1, 1], ['test1', 5, 1], ['test2', 2, 0], ['test3', 4, 0], ['test4', 20, 0]]
['control', 'empty', 'test1', 'test1', 'test1', 'test1', 'test1', 'empty', 'test2', 'test2', 'test3', 'test3', 'test3', 'test3', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4', 'test4']


Map wells to sample names

In [17]:
well_cols = list(range(1,13))


well_rows = ['A','B','C','D','E','F','G','H']

well_dict = {}

for row in well_rows:
    for col in well_cols:
        well_dict[row + str(col)] ='empty'


for i, well in enumerate(well_dict):
    try:
        well_dict[well] = sample_rxn_list[i]
    except:
        well_dict[well] = 'empty'

print(well_dict)


{'A1': 'control', 'A2': 'empty', 'A3': 'test1', 'A4': 'test1', 'A5': 'test1', 'A6': 'test1', 'A7': 'test1', 'A8': 'empty', 'A9': 'test2', 'A10': 'test2', 'A11': 'test3', 'A12': 'test3', 'B1': 'test3', 'B2': 'test3', 'B3': 'test4', 'B4': 'test4', 'B5': 'test4', 'B6': 'test4', 'B7': 'test4', 'B8': 'test4', 'B9': 'test4', 'B10': 'test4', 'B11': 'test4', 'B12': 'test4', 'C1': 'test4', 'C2': 'test4', 'C3': 'test4', 'C4': 'test4', 'C5': 'test4', 'C6': 'test4', 'C7': 'test4', 'C8': 'test4', 'C9': 'test4', 'C10': 'test4', 'C11': 'empty', 'C12': 'empty', 'D1': 'empty', 'D2': 'empty', 'D3': 'empty', 'D4': 'empty', 'D5': 'empty', 'D6': 'empty', 'D7': 'empty', 'D8': 'empty', 'D9': 'empty', 'D10': 'empty', 'D11': 'empty', 'D12': 'empty', 'E1': 'empty', 'E2': 'empty', 'E3': 'empty', 'E4': 'empty', 'E5': 'empty', 'E6': 'empty', 'E7': 'empty', 'E8': 'empty', 'E9': 'empty', 'E10': 'empty', 'E11': 'empty', 'E12': 'empty', 'F1': 'empty', 'F2': 'empty', 'F3': 'empty', 'F4': 'empty', 'F5': 'empty', 'F6': '

Create a dict of row specific lists then split each sample based on well_dict.key into specific row list

In [30]:
rows_dict = {'A':[],'B':[],'C':[],'D':[],'E':[],'F':[],'G':[],'H':[]}


#filling the rows with the well_dict values
for well in well_dict.items():
    for row in rows_dict.items():
        #checkes first letter of well and row
        if well[0][0] == row[0]:
            row[1].append(well[1])
            
#converting the dict to a dataframe
plate_layout = pd.DataFrame(rows_dict)
#transposing the dataframe, going from long to wide
plate_layout = plate_layout.T

#renumber columns to match plate
plate_layout.columns = list(range(1,13))

print(plate_layout)

plate_layout.to_excel('plate_layout.xlsx', engine='openpyxl')

KeyboardInterrupt: Interrupted by user